In [15]:
import numpy as np
import torch
#credit goes to https://blog.floydhub.com/a-beginners-guide-on-recurrent-neural-networks-with-pytorch/
text = ['hey how are you','good i am fine','have a nice day']

chars = set(''.join(text))

int2char = dict(enumerate(chars))

char2int = {char: ind for ind, char in int2char.items()}
print(int2char,",",char2int)

maxlen = len(max(text,key=len))

# Padding

# A simple loop that loops through the list of sentences and adds a ' ' whitespace until the length of
# the sentence matches the length of the longest sentence
for i in range(len(text)):
  while len(text[i])<maxlen:
      text[i] += ' '

{0: 'e', 1: 'm', 2: 'o', 3: 'w', 4: ' ', 5: 'g', 6: 'd', 7: 'h', 8: 'c', 9: 'u', 10: 'a', 11: 'i', 12: 'r', 13: 'y', 14: 'f', 15: 'v', 16: 'n'} , {'e': 0, 'm': 1, 'o': 2, 'w': 3, ' ': 4, 'g': 5, 'd': 6, 'h': 7, 'c': 8, 'u': 9, 'a': 10, 'i': 11, 'r': 12, 'y': 13, 'f': 14, 'v': 15, 'n': 16}


> import numpy as np
> import torch

Basic imports it's very useful.

> chars = set(''.join(text))

the "set function" turns a string into a set (a list with non repeatable elements) it uses the string/character called upon as a separator for each element.
In this case we're turning all of the characters into a set so we use integer identification for each letter and vice versa. (char to int)
It's similar to basic arrays or any sort of unique identification number. it's very useful to achieve what we're trying to do, which is guess the final few characters given only part of a sentence. One alternative I can think of is to iterate through the entire array and add the seperator, but that would take too long, and would be far less readable.

> int2char = dict(enumerate(chars))

2 functions worth eplaining here

### dict()

Dict **is** a function that turns an iterable into a dictionary. In other words, **it's** a hash set that turns an iterable with a key value pair into a dictionary. **For example**, If we had an iterable that already has a kv pair called "example" all we would need to do is call `dict(example)` and we would be able to use the dictonary as a hash map. Another way we could use it, if we didn't have an iterable that was a key value pair, is to used something simlar to named paramaters `dict(x=1,y=2,z=3)` where *x, y and z* are all variables. **Each of them** have their best use case, but of course you should use dict as you would use a hash map. **It's very useful** to use this function like named parameters, although I prefer another method which would use a constructor like in C# (although I can easily get used to this)

### enumerate()

Enumerate **creates** an indexed list. There are iterables in python that, by default, are not enumerated, this function **attempts to** add indexes to each entry into an array. **For example** lets say that we have a set (which by default isn't enumerated) of words that we need to access individually or that need to be modified, we can call enumerate by using `enumerate(x)` to turn set x into an enumerated list. Now we can call `x[y]` in order to access element y from set x. **An Alternative** would be to, of course, iterate over the set, although I'll admit I am not sure if that would work. **I think** it's a workable solution, I can't really think of an alternative as I'm not familar with iteration with python.

> char2int = {char: ind for ind, char in int2char.items()}

## What?
### python for loops

python for loops can be written like: `for x in y:` where "x" is the value inside of iterable y. **it's similar** to a C# foreach loop (`foreach(Type x in y)`) where x is the value and y is the iterable. **It can be used** to iterate through any list. The syntax **is no where near** as straight forward as other syntaxes in this language, although I guess other syntaxes may be more verbose. There is something similar for iterables with a key value pair(`for x,y in z`) where z is an iterable with a key value pair, x is the key and y is the value. For example, given iterable `z = {"one":1,"two":2,"three":3}` the first iteration of `for x,y in z: (line break) print(x,":",y)` would output one:1. **I think** that this code is very confusing and requires knowing how it works in order to be readable, it is brief, but i may just stick to other more clear methods of writing this, or just use comments.

### {char: ind for ind, char in int2char.items()}

This **mess** is a combination of an array literal `{1,2,3}` and the for loop mentioned before. **In this case** we are creating a dictionary that is the inverse of out int2char array. Although this code **is** shorthand for mapping to arrays of equal length together. It's **very useful** albeit very confusing. 


# So what does this do

On a broader scale, this part of the code consists of definitions that we will use in order to simplify the entire process and void writing redundant code



In [16]:
input_seq = []
target_seq = []

for i in range(len(text)):
    input_seq.append(text[i][:-1])

    target_seq.append(text[i][1:])
    print("Input Sequence: {}\nTarget Sequence: {}".format(input_seq[i], target_seq[i]))
for i in range(len(text)):
    input_seq[i] = [char2int[character] for character in input_seq[i]]
    target_seq[i] = [char2int[character] for character in target_seq[i]]

Input Sequence: hey how are yo
Target Sequence: ey how are you
Input Sequence: good i am fine
Target Sequence: ood i am fine 
Input Sequence: have a nice da
Target Sequence: ave a nice day


> text[i][:-1]

### py array slicing

**There are** several parts to array indexing in python the, start, end, and step. The syntax **is** `[start:end:step]`. **For example** if we wanted to take the string `x="ThisThis is a long sentence in someparts of the world.world."` and slice it to get `"This is a long sentence in someparts of the world."` we could slice x into `x[5:-6]`. There's more, if we only want every other letter we can use `x[::2]` or to write it in reverse `x[::-1]`. It's **similar** to substrings in other languages and equally as confusing. **Thankfully** the syntax is shorter and you don't have to deal with the annoyance of typing `substr(5,x.length -1)` every time.

# So what does this do?

It's more utilities that help use achieve our goal of finishing the sentence, It gives us a target sentence and possible inputs. Similar to how you would teach a neural network basic classification.


In [17]:
dict_size = len(char2int)
seq_len = maxlen - 1
batch_size = len(text)

def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    # Creating a multi-dimensional array of zeros with the desired output shape
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
    
    # Replacing the 0 at the relevant character index with a 1 to represent that character
    for i in range(batch_size):
        for u in range(seq_len):
            features[i, u, sequence[i][u]] = 1
    return features

In [18]:
input_seq = one_hot_encode(input_seq,dict_size,seq_len,batch_size)

input_seq = torch.from_numpy(input_seq)
target_seq = torch.Tensor(target_seq)

In [19]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [20]:
class Model(torch.nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = torch.nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = torch.nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        
        batch_size = x.size(0)

        # Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        # We'll send the tensor holding the hidden state to the device we specified earlier as well
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        return hidden

### What does this do

This sets up a recurrent neural network with one layer. Recurrent neural networks require hidden layers as well as the normal input layers.

In [21]:
# Instantiate the model with hyperparameters
model = Model(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1)
# We'll also set the model to the device that we defined earlier (default is CPU)
model.to(device)

# Define hyperparameters
n_epochs = 100
lr=0.01

# Define Loss, Optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

> torch.nn.CrossEntropyLoss()

Cross entropy loss **is** a loss function that measures the difference of probabilities to the desired output.

In [22]:
# Training Run
for epoch in range(1, n_epochs + 1):
    optimizer.zero_grad() # Clears existing gradients from previous epoch
    input_seq.to(device)
    output, hidden = model(input_seq)
    loss = criterion(output, target_seq.view(-1).long())
    loss.backward() # Does backpropagation and calculates gradients
    optimizer.step() # Updates the weights accordingly
    
    if epoch%10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))
        

Epoch: 10/100............. Loss: 2.4254
Epoch: 20/100............. Loss: 2.1465
Epoch: 30/100............. Loss: 1.7464
Epoch: 40/100............. Loss: 1.3114
Epoch: 50/100............. Loss: 0.9258
Epoch: 60/100............. Loss: 0.6227
Epoch: 70/100............. Loss: 0.4086
Epoch: 80/100............. Loss: 0.2724
Epoch: 90/100............. Loss: 0.1946
Epoch: 100/100............. Loss: 0.1492


In [23]:
# This function takes in the model and character as arguments and returns the next character prediction and hidden state
def predict(model, character):
    # One-hot encoding our input to fit into the model
    character = np.array([[char2int[c] for c in character]])
    character = one_hot_encode(character, dict_size, character.shape[1], 1)
    character = torch.from_numpy(character)
    character.to(device)
    
    out, hidden = model(character)

    prob = torch.nn.functional.softmax(out[-1], dim=0).data
    # Taking the class with the highest probability score from the output
    char_ind = torch.max(prob, dim=0)[1].item()

    return int2char[char_ind], hidden
# This function takes the desired output length and input characters as arguments, returning the produced sentence
def sample(model, out_len, start='hey'):
    model.eval() # eval mode
    start = start.lower()
    # First off, run through the starting characters
    chars = [ch for ch in start]
    size = out_len - len(chars)
    # Now pass in the previous characters and get a new one
    for ii in range(size):
        char, h = predict(model, chars)
        chars.append(char)

    return ''.join(chars)

In [32]:
sample(model, 15, 'good')

'good i am fine '